In [21]:
# reload magics
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datasets
from project_modules.io import load_dataset_to_df
from project_modules.classifcation import classify_MP,getXY, boruta_fs
from project_modules.utils import MPutils
from sklearn.ensemble import RandomForestClassifier
import multiprocessing
from sklearn.model_selection import cross_val_score
import cupy as cp
from datetime import datetime

from tqdm.notebook import tqdm
from copy import deepcopy

# from project_modules.utils import get_logger
# logger = get_logger("log-data-combine-split.log")
# # read the parameter file

# from project_modules.utils import read_parameters
# parms = read_parameters("/Users/david/projects/lc-project-data/project.yaml")

pd.options.display.max_columns = None
pd.options.display.max_rows = 50

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
lScorersBinary = [
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    # "average_precision",
    "NPV",
    "PPV",
    # "neg_mean_squared_error",
]
lResCol = [
    "Title",
    "cv",
    "param_clf",
    "param_clf__max_depth",
    "param_clf__n_estimators",
    "param_clf__random_state",
    # "param_clf__max_iter",
    "mean_test_accuracy",
    "mean_test_balanced_accuracy",
    "mean_test_roc_auc",
    "mean_test_f1",
    "mean_test_recall",
    "mean_test_sensitivity",
    "mean_test_specificity",
    "mean_test_precision",
    "mean_test_NPV",
    "mean_test_PPV",
]

In [23]:
site_name_dict = {1: "LDN", 2: "MTL", 3: "SAN", 4: "RIO", 5: "LUS", 6: "CA", 7: "NA"}
status_name_dict = {0: "HC", 1: "LC_NEG", 2: "LC_POS"}

# Combine Results

In [24]:
import glob

path = "OUTPUT/MP/05-classifiers/DataV3"
files = glob.glob(path + '/*/CA*.csv')
# files = files+glob.glob(path + "/*/FS*.csv")
files = files + glob.glob(path + "/*/Val*.csv")
files = files+glob.glob(path+'/*/CLFRun*.csv')
files = [x for x in files if 'selcol' not in x.lower()]

print(files)
print(len(files))

['OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC_SITE5/CA_dfcmpltPreLC_SITE5_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC_SITE5/CA_dfcmpltPreLC_SITE5_Full.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmplt_SITE2/CA_dfcmplt_SITE2_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmplt_SITE2/CA_dfcmplt_SITE2_Full.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC3_SITE3/CA_dfcmpltPreLC3_SITE3_Full.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC3_SITE3/CA_dfcmpltPreLC3_SITE3_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC3_SITE4/CA_dfcmpltPreLC3_SITE4_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC3_SITE4/CA_dfcmpltPreLC3_SITE4_Full.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC3_SITE5/CA_dfcmpltPreLC3_SITE5_Holdout.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC3_SITE5/CA_dfcmpltPreLC3_SITE5_Full.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC2_SITE2/CA_dfcmpltPreLC2_SITE2_Full.csv', 'OUTPUT/MP/05-classifiers/DataV3/dfcmpltPreLC2_SITE2/CA

In [25]:
ldf = []
for i in files:
    ldf.append(pd.read_csv(i))

In [26]:
dfRes = pd.concat(ldf)

In [27]:
dfRes

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_balanced_accuracy,split1_test_balanced_accuracy,split2_test_balanced_accuracy,split3_test_balanced_accuracy,split4_test_balanced_accuracy,mean_test_balanced_accuracy,std_test_balanced_accuracy,rank_test_balanced_accuracy,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,split3_test_roc_auc,split4_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_sensitivity,split1_test_sensitivity,split2_test_sensitivity,split3_test_sensitivity,split4_test_sensitivity,mean_test_sensitivity,std_test_sensitivity,rank_test_sensitivity,split0_test_specificity,split1_test_specificity,split2_test_specificity,split3_test_specificity,split4_test_specificity,mean_test_specificity,std_test_specificity,rank_test_specificity,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_NPV,split1_test_NPV,split2_test_NPV,split3_test_NPV,split4_test_NPV,mean_test_NPV,std_test_NPV,rank_test_NPV,split0_test_PPV,split1_test_PPV,split2_test_PPV,split3_test_PPV,split4_test_PPV,mean_test_PPV,std_test_PPV,rank_test_PPV,cv,Title,RunType,MainDataset,date,classifier,FeatureSet,brt_nTrue,brt_nTop,brt_md,brt_params
0,0.090703,0.014567,0.020051,0.003527,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,"{'clf': RandomForestClassifier(), 'clf__max_de...",0.775510,0.812500,0.770833,0.770833,0.770833,0.780102,0.016300,1,0.500000,0.550000,0.500000,0.500000,0.500000,0.510000,0.020000,33,0.590909,0.786842,0.658477,0.717445,0.710074,0.692749,0.065283,2,0.000000,0.181818,0.000000,0.000000,0.000000,0.036364,0.072727,34,0.000000,0.100000,0.000000,0.000000,0.000000,0.020000,0.040000,34,0.000000,0.100000,0.000000,0.000000,0.000000,0.020000,0.040000,34,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1,0.000000,1.000000,0.000000,0.000000,0.000000,0.200000,0.400000,34,0.775510,0.808511,0.770833,0.770833,0.770833,0.779304,0.014715,33,0.000000,1.000000,0.000000,0.000000,0.000000,0.200000,0.400000,34,5,dfcmpltPreLC_SITE5_Holdout,Holdout,dfcmpltPreLC_SITE5,<built-in method today of type object at 0x7ad...,NaN,NaN,NaN,NaN,NaN,NaN
1,0.891907,0.154319,0.081418,0.001751,RandomForestClassifier(),3.0,1000.0,42.0,NaN,NaN,"{'clf': RandomForestClassifier(), 'clf__max_de...",0.775510,0.812500,0.770833,0.770833,0.770833,0.780102,0.016300,1,0.500000,0.550000,0.500000,0.500000,0.500000,0.510000,0.020000,33,0.593301,0.784211,0.616708,0.714988,0.756757,0.693193,0.075668,1,0.000000,0.181818,0.000000,0.000000,0.000000,0.036364,0.072727,34,0.000000,0.100000,0.000000,0.000000,0.000000,0.020000,0.040000,34,0.000000,0.100000,0.000000,0.000000,0.000000,0.020000,0.040000,34,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1,0.000000,1.000000,0.000000,0.000000,0.000000,0.200000,0.400000,34,0.775510,0.808511,0.770833,0.770833,0.770833,0.779304,0.014715,33,0.000000,1.000000,0.000000,0.000000,0.000000,0.200000,0.400000,34,5,dfcmpltPreLC_SITE5_Holdout,Holdout,dfcmpltPreLC_SITE5,<built-in method today of type object at 0x7ad...,NaN,NaN,NaN,NaN,NaN,NaN
2,0.106432,0.004907,0.022866,0.000483,RandomForestClassifier(),6.0,100.0,42.0,NaN,NaN,"{'clf': RandomForestClassifier(), 'clf__max_de...",0.775510,0.812500,0.770833,0.770833,0.770833,0.780102,0.016300,1,0.500000,0.550000,0.500000,0.500000,0.500000,0.510000,0.020000,33,0.578947,0.

In [28]:
dfRes.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_clf', 'param_clf__max_depth', 'param_clf__n_estimators',
       'param_clf__random_state', 'param_clf__max_iter',
       'param_clf__learning_rate',
       ...
       'Title', 'RunType', 'MainDataset', 'date', 'classifier', 'FeatureSet',
       'brt_nTrue', 'brt_nTop', 'brt_md', 'brt_params'],
      dtype='object', length=102)

In [29]:
def changeSite(x):
    if pd.isnull(x):
        return np.nan
    if 'SITE7' in x:
        return 'NorthAmerica'
    if 'SITE6' in x:
        return 'Canada'
    if 'SITE5' in x:
        return 'LUS'
    if 'SITE4' in x:
        return 'RIO'
    if 'SITE3' in x:
        return 'SAN'
    if 'SITE2' in x:
        return 'MTL'
    if 'SITE1' in x:
        return 'LDN'
    return "Global"

In [30]:
dfRes

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_balanced_accuracy,split1_test_balanced_accuracy,split2_test_balanced_accuracy,split3_test_balanced_accuracy,split4_test_balanced_accuracy,mean_test_balanced_accuracy,std_test_balanced_accuracy,rank_test_balanced_accuracy,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,split3_test_roc_auc,split4_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_sensitivity,split1_test_sensitivity,split2_test_sensitivity,split3_test_sensitivity,split4_test_sensitivity,mean_test_sensitivity,std_test_sensitivity,rank_test_sensitivity,split0_test_specificity,split1_test_specificity,split2_test_specificity,split3_test_specificity,split4_test_specificity,mean_test_specificity,std_test_specificity,rank_test_specificity,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_NPV,split1_test_NPV,split2_test_NPV,split3_test_NPV,split4_test_NPV,mean_test_NPV,std_test_NPV,rank_test_NPV,split0_test_PPV,split1_test_PPV,split2_test_PPV,split3_test_PPV,split4_test_PPV,mean_test_PPV,std_test_PPV,rank_test_PPV,cv,Title,RunType,MainDataset,date,classifier,FeatureSet,brt_nTrue,brt_nTop,brt_md,brt_params
0,0.090703,0.014567,0.020051,0.003527,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,"{'clf': RandomForestClassifier(), 'clf__max_de...",0.775510,0.812500,0.770833,0.770833,0.770833,0.780102,0.016300,1,0.500000,0.550000,0.500000,0.500000,0.500000,0.510000,0.020000,33,0.590909,0.786842,0.658477,0.717445,0.710074,0.692749,0.065283,2,0.000000,0.181818,0.000000,0.000000,0.000000,0.036364,0.072727,34,0.000000,0.100000,0.000000,0.000000,0.000000,0.020000,0.040000,34,0.000000,0.100000,0.000000,0.000000,0.000000,0.020000,0.040000,34,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1,0.000000,1.000000,0.000000,0.000000,0.000000,0.200000,0.400000,34,0.775510,0.808511,0.770833,0.770833,0.770833,0.779304,0.014715,33,0.000000,1.000000,0.000000,0.000000,0.000000,0.200000,0.400000,34,5,dfcmpltPreLC_SITE5_Holdout,Holdout,dfcmpltPreLC_SITE5,<built-in method today of type object at 0x7ad...,NaN,NaN,NaN,NaN,NaN,NaN
1,0.891907,0.154319,0.081418,0.001751,RandomForestClassifier(),3.0,1000.0,42.0,NaN,NaN,"{'clf': RandomForestClassifier(), 'clf__max_de...",0.775510,0.812500,0.770833,0.770833,0.770833,0.780102,0.016300,1,0.500000,0.550000,0.500000,0.500000,0.500000,0.510000,0.020000,33,0.593301,0.784211,0.616708,0.714988,0.756757,0.693193,0.075668,1,0.000000,0.181818,0.000000,0.000000,0.000000,0.036364,0.072727,34,0.000000,0.100000,0.000000,0.000000,0.000000,0.020000,0.040000,34,0.000000,0.100000,0.000000,0.000000,0.000000,0.020000,0.040000,34,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1,0.000000,1.000000,0.000000,0.000000,0.000000,0.200000,0.400000,34,0.775510,0.808511,0.770833,0.770833,0.770833,0.779304,0.014715,33,0.000000,1.000000,0.000000,0.000000,0.000000,0.200000,0.400000,34,5,dfcmpltPreLC_SITE5_Holdout,Holdout,dfcmpltPreLC_SITE5,<built-in method today of type object at 0x7ad...,NaN,NaN,NaN,NaN,NaN,NaN
2,0.106432,0.004907,0.022866,0.000483,RandomForestClassifier(),6.0,100.0,42.0,NaN,NaN,"{'clf': RandomForestClassifier(), 'clf__max_de...",0.775510,0.812500,0.770833,0.770833,0.770833,0.780102,0.016300,1,0.500000,0.550000,0.500000,0.500000,0.500000,0.510000,0.020000,33,0.578947,0.

In [31]:
dfRes["classifier"] = dfRes["param_clf"].apply(lambda x: x.__str__().split("(")[0])
dfRes['site'] = dfRes['MainDataset'].apply(changeSite)
lColFirst = [
    "MainDataset",
    "Title",
    'site',
    "date",
    "classifier",
    'RunType',
    "cv",
    'FeatureSet',
    "brt_nTrue",
    "brt_nTop",
    'brt_md',
]
dfRes = MPutils.reorder_columns(dfRes, lColFirst)
dfRes.to_csv("OUTPUT/MP/05-classifiers/DataV3/CLFRunCombined.csv", index=False)
dfResOrg = deepcopy(dfRes)
lColsSel = lColFirst + [
    x
    for x in dfRes
    if (x.startswith("mean_test") or x.startswith("param_") or x.startswith("brt_"))
    and x not in lColFirst
]
dfRes[lColsSel].to_csv("OUTPUT/MP/05-classifiers/DataV3/CLFRunCombined_SelCol.csv")

# Go through Data and Simplify for inspection

In [33]:
dfRes = dfRes[lColsSel]

In [34]:
dfRes

,MainDataset,Title,site,date,classifier,RunType,cv,FeatureSet,brt_nTrue,brt_nTop,brt_md,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,mean_test_accuracy,mean_test_balanced_accuracy,mean_test_roc_auc,mean_test_f1,mean_test_recall,mean_test_sensitivity,mean_test_specificity,mean_test_precision,mean_test_NPV,mean_test_PPV,brt_params
0,dfcmpltPreLC_SITE5,dfcmpltPreLC_SITE5_Holdout,LUS,<built-in method today of type object at 0x7ad...,RandomForestClassifier,Holdout,5,NaN,NaN,NaN,NaN,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,0.780102,0.510000,0.692749,0.036364,0.020000,0.020000,1.000000,0.200000,0.779304,0.200000,NaN
1,dfcmpltPreLC_SITE5,dfcmpltPreLC_SITE5_Holdout,LUS,<built-in method today of type object at 0x7ad...,RandomForestClassifier,Holdout,5,NaN,NaN,NaN,NaN,RandomForestClassifier(),3.0,1000.0,42.0,NaN,NaN,0.780102,0.510000,0.693193,0.036364,0.020000,0.020000,1.000000,0.200000,0.779304,0.200000,NaN
2,dfcmpltPreLC_SITE5,dfcmpltPreLC_SITE5_Holdout,LUS,<built-in method today of type object at 0x7ad...,RandomForestClassifier,Holdout,5,NaN,NaN,NaN,NaN,RandomForestClassifier(),6.0,100.0,42.0,NaN,NaN,0.780102,0.510000,0.652312,0.036364,0.020000,0.020000,1.000000,0.200000,0.779304,0.200000,NaN
3,dfcmpltPreLC_SITE5,dfcmpltPreLC_SITE5_Holdout,LUS,<built-in method today of type object at 0x7ad...,RandomForestClassifier,Holdout,5,NaN,NaN,NaN,NaN,RandomForestClassifier(),6.0,1000.0,42.0,NaN,NaN,0.780102,0.510000,0.674936,0.036364,0.020000,0.020000,1.000000,0.200000,0.779304,0.200000,NaN
4,dfcmpltPreLC_SITE5,dfcmpltPreLC_SITE5_Holdout,LUS,<built-in method today of type object at 0x7ad...,RandomForestClassifier,Holdout,5,NaN,NaN,NaN,NaN,RandomForestClassifier(),10.0,100.0,42.0,NaN,NaN,0.771854,0.511054,0.677128,0.064935,0.038182,0.038182,0.983926,0.266667,0.779758,0.266667,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,dfcmpltPreLC2_SITE6,dfcmpltPreLC2_SITE6_Boruta_Top,Canada,2024-05-01 21:34:03.895196,XGBClassifier,Boruta_Top,5,NaN,2.0,6.0,7.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,1.0,0.634375,0.564676,0.603757,0.392622,0.370732,0.370732,0.758621,0.419044,0.719307,0.419044,T1000_itrr500_th100_topR5
324,dfcmpltPreLC2_SITE6,dfcmpltPreLC2_SITE6_Boruta_Top,Canada,2024-05-01 21:34:03.895196,XGBClassifier,Boruta_Top,5,NaN,2.0,6.0,7.0,"XGBClassifier(base_score=None, booster=None, c...",15.0,100.0,42.0,NaN,1.0,0.639062,0.575862,0.606672,0.415253,0.400000,0.400000,0.751724,0.433285,0.726611,0.433285,T1000_itrr500_th100_topR5
325,dfcmpltPreLC2_SITE6,dfcmpltPreLC2_SITE6_Boruta_Top,Canada,2024-05-01 21:34:03.895196,XGBClassifier,Boruta_Top,5,NaN,2.0,6.0,7.0,"XGBClassifier(base_score=None, booster=None, c...",15.0,1000.0,42.0,NaN,1.0,0.634375,0.569835,0.600561,0.404921,0.390244,0.390244,0.749425,0.422719,0.723220,0.422719,T1000_itrr500_th100_topR5
326,dfcmpltPreLC2_SITE6,dfcmpltPreLC2_SITE6_Boruta_Top,Canada,2024-05-01 21:34:03.895196,XGBClassifier,Boruta_Top,5,NaN,2.0,6.0,7.0,"XGBClassifier(base_score=None, booster=None, c...",NaN,100.0,42.0,NaN,1.0,0.628125,0.562658,0.612700,0.395360,0.380488,0.380488,0.744828,0.414113,0.718490,0.414113,T1000_itrr500_th100_topR5


In [35]:
lSelMainData = [
"dfcmpltPreLC3",
"dfcmpltPreLC3_SITE1",
"dfcmpltPreLC3_SITE2",
"dfcmpltPreLC3_SITE3",
"dfcmpltPreLC3_SITE4",
"dfcmpltPreLC3_SITE5",
"dfcmpltPreLC3_SITE6",
"dfcmpltPreLC3_SITE7",
]

lPrettyCols =     [
        "MainDataset",
        "RunType",
        "classifier",
        "brt_nTrue",
        "brt_nTop",
        "accuracy",
        "balanced_accuracy",
        "roc_auc",
        "f1",
        "recall",
        "sensitivity",
        "specificity",
        "precision",
        "NPV",
        "PPV",
        "brt_md",
        'brt_params'
    ]
lPrettyColsSite = [
    "MainDataset",
    "RunType",
    'site',
    "classifier",
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    "NPV",
    "PPV",
]
s_brt = 'and (RunType=="Boruta_True" or RunType=="Boruta_Top")'
s_RF10 = 'classifier=="RandomForestClassifier" and param_clf__n_estimators==1000 and param_clf__max_depth==10'
s_RFXGB10 = 'param_clf__n_estimators==1000 and param_clf__max_depth==10'
s_CMPLT = 'MainDataset=="dfcmpltPreLC"'
s_val = 'RunType=="HoldoutVal"'

In [36]:
dfRes = dfRes[dfRes['MainDataset'].isin(lSelMainData)]
dfRes.columns = [x.replace('mean_test_','') for x in dfRes.columns]

# Inspect Original Holdout
- May have duplicates

In [41]:
dfRes.query(f'{s_RF10} and RunType=="Holdout"').sort_values("balanced_accuracy", ascending=False)[lPrettyColsSite].T

,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
MainDataset,dfcmpltPreLC2_SITE4,dfcmpltPreLC2_SITE4,dfcmpltPreLC2_SITE1,dfcmpltPreLC2_SITE1,dfcmpltPreLC2_SITE3,dfcmpltPreLC2_SITE3,dfcmpltPreLC2,dfcmpltPreLC2,dfcmpltPreLC2_SITE7,dfcmpltPreLC2_SITE7,dfcmpltPreLC2_SITE6,dfcmpltPreLC2_SITE6,dfcmpltPreLC2_SITE2,dfcmpltPreLC2_SITE2,dfcmpltPreLC2_SITE5,dfcmpltPreLC2_SITE5
RunType,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout,Holdout
site,RIO,RIO,LDN,LDN,SAN,SAN,Global,Global,NorthAmerica,NorthAmerica,Canada,Canada,MTL,MTL,LUS,LUS
classifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier
accuracy,1.0,1.0,0.878788,0.878788,0.865934,0.865934,0.766997,0.766997,0.743582,0.743582,0.721383,0.721383,0.706116,0.706116,0.77602,0.77602
balanced_accuracy,1.0,1.0,0.877857,0.877857,0.826061,0.826061,0.724582,0.724582,0.667499,0.667499,0.596697,0.596697,0.563611,0.563611,0.507368,0.507368
roc_auc,1.0,1.0,0.96,0.96,0.916364,0.916364,0.805964,0.805964,0.792362,0.792362,0.747603,0.747603,0.714913,0.714913,0.67889,0.67889
f1,1.0,1.0,0.85,0.85,0.911104,0.911104,0.636448,0.636448,0.52093,0.52093,0.364115,0.364115,0.277852,0.277852,0.036364,0.036364
recall,1.0,1.0,0.87,0.87,0.885455,0.885455,0.511159,0.511159,0.384087,0.384087,0.24878,0.24878,0.17988,0.17988,0.02,0.02
sensitivity,1.0,1.0,0.87,0.87,0.885455,0.885455,0.511159,0.511159,0.384087,0.384087,0.24878,0.24878,0.17988,0.17988,0.02,0.02


# Per site - Best Results

In [37]:
dfRes.query(f'{s_RF10} and RunType=="Boruta_True"').sort_values('balanced_accuracy',ascending=False)

,MainDataset,Title,site,date,classifier,RunType,cv,FeatureSet,brt_nTrue,brt_nTop,brt_md,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_params
87,dfcmpltPreLC3_SITE4,dfcmpltPreLC3_SITE4_Boruta_True,RIO,2024-05-03 14:54:12.395441,RandomForestClassifier,Boruta_True,5,NaN,17.0,21.0,3.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.981048,0.946667,0.996923,0.988819,1.000000,1.000000,0.893333,0.978307,1.000000,0.978307,T1000_itrr500_th100_topR5
169,dfcmpltPreLC3_SITE4,dfcmpltPreLC3_SITE4_Boruta_True,RIO,2024-05-03 14:54:12.395441,RandomForestClassifier,Boruta_True,5,NaN,11.0,15.0,5.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.981048,0.946667,0.996923,0.988819,1.000000,1.000000,0.893333,0.978307,1.000000,0.978307,T1000_itrr500_th100_topR5
251,dfcmpltPreLC3_SITE4,dfcmpltPreLC3_SITE4_Boruta_True,RIO,2024-05-03 14:54:12.395441,RandomForestClassifier,Boruta_True,5,NaN,10.0,14.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.981048,0.946667,0.996923,0.988819,1.000000,1.000000,0.893333,0.978307,1.000000,0.978307,T1000_itrr500_th100_topR5
87,dfcmpltPreLC3_SITE3,dfcmpltPreLC3_SITE3_Boruta_True,SAN,2024-05-03 14:39:06.791737,RandomForestClassifier,Boruta_True,5,NaN,13.0,18.0,3.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.910989,0.855152,0.950909,0.942733,0.943636,0.943636,0.766667,0.943333,0.833333,0.943333,T1000_itrr500_th100_topR5
169,dfcmpltPreLC3_SITE3,dfcmpltPreLC3_SITE3_Boruta_True,SAN,2024-05-03 14:39:06.791737,RandomForestClassifier,Boruta_True,5,NaN,11.0,16.0,5.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.910989,0.855152,0.950909,0.942733,0.943636,0.943636,0.766667,0.943333,0.833333,0.943333,T1000_itrr500_th100_topR5
251,dfcmpltPreLC3_SITE3,dfcmpltPreLC3_SITE3_Boruta_True,SAN,2024-05-03 14:39:06.791737,RandomForestClassifier,Boruta_True,5,NaN,5.0,9.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.910989,0.855152,0.963030,0.942733,0.943636,0.943636,0.766667,0.943333,0.833333,0.943333,T1000_itrr500_th100_topR5
169,dfcmpltPreLC3_SITE1,dfcmpltPreLC3_SITE1_Boruta_True,LDN,2024-05-03 14:28:43.788349,RandomForestClassifier,Boruta_True,5,NaN,7.0,11.0,5.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.786364,0.785000,0.895714,0.745671,0.770000,0.770000,0.800000,0.772381,0.847222,0.772381,T1000_itrr500_th100_topR5
251,dfcmpltPreLC3_SITE1,dfcmpltPreLC3_SITE1_Boruta_True,LDN,2024-05-03 14:28:43.788349,RandomForestClassifier,Boruta_True,5,NaN,6.0,9.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.768182,0.770714,0.881429,0.736580,0.770000,0.770000,0.771429,0.761667,0.830556,0.761667,T1000_itrr500_th100_topR5
87,dfcmpltPreLC3_SITE1,dfcmpltPreLC3_SITE1_Boruta_True,LDN,2024-05-03 14:28:43.788349,RandomForestClassifier,Boruta_True,5,NaN,16.0,21.0,3.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.769697,0.754286,0.872857,0.711429,0.680000,0.680000,0.828571,0.776667,0.793413,0.776667,T1000_itrr500_th100_topR5
99,dfcmpltPreLC3,dfcmpltPreLC3_Boruta_True,Global,2024-05-03 14:26:18.647207,RandomForestClassifier,Boruta_True,5,NaN,26.0,31.0,3.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.727680,0.697416,0.766245,0.615409,0.550392,0.550392,0.844441,0.700839,0.740830,0.700839,T1000_itrr500_th100_topR5


In [17]:
dfTmp = dfRes.query(f'{s_RF10} and RunType=="Boruta_True" and {s_RF10}').sort_values('balanced_accuracy',ascending=False)
lTopBrt = []
for i in lSelMainData:
    dfTmpTop = dfTmp.query(f'MainDataset=="{i}"').sort_values('balanced_accuracy',ascending=False)
    lTopBrt.append(dfTmpTop.iloc[0,:])

In [18]:
pd.DataFrame(lTopBrt)[lPrettyColsSite+['brt_md','brt_nTrue','Title']].sort_values('balanced_accuracy',ascending=False)

,MainDataset,RunType,site,classifier,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md,brt_nTrue,Title
87,dfcmpltPreLC3_SITE4,Boruta_True,RIO,RandomForestClassifier,0.981048,0.946667,0.996923,0.988819,1.000000,1.000000,0.893333,0.978307,1.000000,0.978307,3.0,17.0,dfcmpltPreLC3_SITE4_Boruta_True
87,dfcmpltPreLC3_SITE3,Boruta_True,SAN,RandomForestClassifier,0.910989,0.855152,0.950909,0.942733,0.943636,0.943636,0.766667,0.943333,0.833333,0.943333,3.0,13.0,dfcmpltPreLC3_SITE3_Boruta_True
169,dfcmpltPreLC3_SITE1,Boruta_True,LDN,RandomForestClassifier,0.786364,0.785000,0.895714,0.745671,0.770000,0.770000,0.800000,0.772381,0.847222,0.772381,5.0,7.0,dfcmpltPreLC3_SITE1_Boruta_True
99,dfcmpltPreLC3,Boruta_True,Global,RandomForestClassifier,0.727680,0.697416,0.766245,0.615409,0.550392,0.550392,0.844441,0.700839,0.740830,0.700839,3.0,26.0,dfcmpltPreLC3_Boruta_True
87,dfcmpltPreLC3_SITE7,Boruta_True,NorthAmerica,RandomForestClassifier,0.721803,0.658295,0.772777,0.527621,0.432471,0.432471,0.884120,0.680391,0.735214,0.680391,3.0,26.0,dfcmpltPreLC3_SITE7_Boruta_True
87,dfcmpltPreLC3_SITE2,Boruta_True,MTL,RandomForestClassifier,0.715757,0.613750,0.712224,0.429111,0.350000,0.350000,0.877500,0.556377,0.753400,0.556377,3.0,14.0,dfcmpltPreLC3_SITE2_Boruta_True
87,dfcmpltPreLC3_SITE6,Boruta_True,Canada,RandomForestClassifier,0.714511,0.610597,0.723313,0.420857,0.331538,0.331538,0.889655,0.582788,0.744335,0.582788,3.0,20.0,dfcmpltPreLC3_SITE6_Boruta_True
87,dfcmpltPreLC3_SITE5,Boruta_True,LUS,RandomForestClassifier,0.684609,0.498817,0.520663,0.178355,0.163636,0.163636,0.833997,0.200260,0.776252,0.200260,3.0,4.0,dfcmpltPreLC3_SITE5_Boruta_True


In [20]:
pd.DataFrame(lTopBrt)[['brt_md','brt_nTrue','Title']+lPrettyColsSite].sort_values('MainDataset',ascending=False)

,brt_md,brt_nTrue,Title,MainDataset,RunType,site,classifier,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV
87,3.0,26.0,dfcmpltPreLC3_SITE7_Boruta_True,dfcmpltPreLC3_SITE7,Boruta_True,NorthAmerica,RandomForestClassifier,0.721803,0.658295,0.772777,0.527621,0.432471,0.432471,0.884120,0.680391,0.735214,0.680391
87,3.0,20.0,dfcmpltPreLC3_SITE6_Boruta_True,dfcmpltPreLC3_SITE6,Boruta_True,Canada,RandomForestClassifier,0.714511,0.610597,0.723313,0.420857,0.331538,0.331538,0.889655,0.582788,0.744335,0.582788
87,3.0,4.0,dfcmpltPreLC3_SITE5_Boruta_True,dfcmpltPreLC3_SITE5,Boruta_True,LUS,RandomForestClassifier,0.684609,0.498817,0.520663,0.178355,0.163636,0.163636,0.833997,0.200260,0.776252,0.200260
87,3.0,17.0,dfcmpltPreLC3_SITE4_Boruta_True,dfcmpltPreLC3_SITE4,Boruta_True,RIO,RandomForestClassifier,0.981048,0.946667,0.996923,0.988819,1.000000,1.000000,0.893333,0.978307,1.000000,0.978307
87,3.0,13.0,dfcmpltPreLC3_SITE3_Boruta_True,dfcmpltPreLC3_SITE3,Boruta_True,SAN,RandomForestClassifier,0.910989,0.855152,0.950909,0.942733,0.943636,0.943636,0.766667,0.943333,0.833333,0.943333
87,3.0,14.0,dfcmpltPreLC3_SITE2_Boruta_True,dfcmpltPreLC3_SITE2,Boruta_True,MTL,RandomForestClassifier,0.715757,0.613750,0.712224,0.429111,0.350000,0.350000,0.877500,0.556377,0.753400,0.556377
169,5.0,7.0,dfcmpltPreLC3_SITE1_Boruta_True,dfcmpltPreLC3_SITE1,Boruta_True,LDN,RandomForestClassifier,0.786364,0.785000,0.895714,0.745671,0.770000,0.770000,0.800000,0.772381,0.847222,0.772381
99,3.0,26.0,dfcmpltPreLC3_Boruta_True,dfcmpltPreLC3,Boruta_True,Global,RandomForestClassifier,0.727680,0.697416,0.766245,0.615409,0.550392,0.550392,0.844441,0.700839,0.740830,0.700839


In [47]:
# # Create the ExcelWriter object
writer = pd.ExcelWriter('../OUTPUTS/prelim_preLongCOVID2_Brt_columns.xlsx', engine='xlsxwriter')
pd.DataFrame(lTopBrt)[lPrettyColsSite+['brt_md','brt_nTrue','Title']].sort_values('balanced_accuracy',ascending=False).to_excel(writer,sheet_name='Results')

for i,d in pd.DataFrame(lTopBrt).iterrows():
    tmpdataset = d['MainDataset']
    md = int(d['brt_md'])
    f = f'OUTPUT/MP/05-classifiers/DataV3/{tmpdataset}/FS_{tmpdataset}_Boruta_T1000_itrr500_th100_topR5_MD{md}.csv'
    dfeat = pd.read_csv(f)
    MPutils.getTrueFeatList(f,asDF=True).to_excel(writer, sheet_name=d['site'])
    


    # Write each dataframe to a different sheet
    # df1.to_excel(writer, sheet_name=d['site'])

#     # Save the Excel file
writer.close()


In [175]:
MPutils.getTrueFeatList(f,asDF=True)

,Rank,TrueFeat,Feat
0,1,True,age
1,1,True,VAX-vaccinated
2,1,True,CD-mode_of_diag
3,1,True,MED-Respiratory-tract-agents
4,1,True,SYMPT-chest_pain___1
6,1,True,SYMPT-cough___1
7,1,True,SYMPT-diarrhea___1
8,1,True,SYMPT-extremity_weakness_or_numb___1
9,1,True,SYMPT-joint_pain_arthralgia___1
10,1,True,SYMPT-loss_of_taste_lost_of_smel___1


In [160]:
pd.DataFrame(lTopBrt)[lPrettyColsSite+['brt_md','brt_nTrue']].sort_values('balanced_accuracy',ascending=False).to_csv('../OUTPUTS/prelim_preLongCOVID_Brt.csv',index=False)

In [146]:
dfTmpTop

,MainDataset,Title,site,date,classifier,RunType,cv,FeatureSet,brt_nTrue,brt_nTop,brt_md,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_params
87,dfcmpltPreLC_SITE6,dfcmpltPreLC_SITE6_Boruta_True,Canada,2024-05-01 01:15:47.719266,RandomForestClassifier,Boruta_True,5,NaN,20.0,24.0,3.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.725000,0.628764,0.728988,0.455713,0.360976,0.360976,0.896552,0.624638,0.748745,0.624638,T1000_itrr500_th100_topR5
169,dfcmpltPreLC_SITE6,dfcmpltPreLC_SITE6_Boruta_True,Canada,2024-05-01 01:15:47.719266,RandomForestClassifier,Boruta_True,5,NaN,7.0,11.0,5.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.660937,0.586796,0.640398,0.416736,0.380488,0.380488,0.793103,0.461775,0.731371,0.461775,T1000_itrr500_th100_topR5
251,dfcmpltPreLC_SITE6,dfcmpltPreLC_SITE6_Boruta_True,Canada,2024-05-01 01:15:47.719266,RandomForestClassifier,Boruta_True,5,NaN,2.0,6.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.643750,0.559966,0.605607,0.368199,0.326829,0.326829,0.793103,0.428030,0.714564,0.428030,T1000_itrr500_th100_topR5


In [147]:
lTopBrt

[MainDataset                               dfcmpltPreLC
 Title                         dfcmpltPreLC_Boruta_True
 site                                            Global
 date                        2024-05-01 00:09:32.529785
 classifier                      RandomForestClassifier
 RunType                                    Boruta_True
 cv                                                   5
 FeatureSet                                         NaN
 brt_nTrue                                         33.0
 brt_nTop                                          37.0
 brt_md                                             3.0
 param_clf                     RandomForestClassifier()
 param_clf__max_depth                              10.0
 param_clf__n_estimators                         1000.0
 param_clf__random_state                           42.0
 param_clf__max_iter                                NaN
 param_clf__learning_rate                           NaN
 accuracy                                      0

In [137]:
pd.concat(lTopBrt)

MainDataset                  dfcmpltPreLC
Title            dfcmpltPreLC_Boruta_True
site                               Global
date           2024-05-01 00:09:32.529785
classifier         RandomForestClassifier
                          ...            
specificity                      0.896552
precision                        0.624638
NPV                              0.748745
PPV                              0.624638
brt_params      T1000_itrr500_th100_topR5
Length: 224, dtype: object

In [133]:
dfTmpTop

,MainDataset,Title,site,date,classifier,RunType,cv,FeatureSet,brt_nTrue,brt_nTop,brt_md,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_params
87,dfcmpltPreLC_SITE6,dfcmpltPreLC_SITE6_Boruta_True,Canada,2024-05-01 01:15:47.719266,RandomForestClassifier,Boruta_True,5,NaN,20.0,24.0,3.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.725000,0.628764,0.728988,0.455713,0.360976,0.360976,0.896552,0.624638,0.748745,0.624638,T1000_itrr500_th100_topR5
169,dfcmpltPreLC_SITE6,dfcmpltPreLC_SITE6_Boruta_True,Canada,2024-05-01 01:15:47.719266,RandomForestClassifier,Boruta_True,5,NaN,7.0,11.0,5.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.660937,0.586796,0.640398,0.416736,0.380488,0.380488,0.793103,0.461775,0.731371,0.461775,T1000_itrr500_th100_topR5
251,dfcmpltPreLC_SITE6,dfcmpltPreLC_SITE6_Boruta_True,Canada,2024-05-01 01:15:47.719266,RandomForestClassifier,Boruta_True,5,NaN,2.0,6.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.643750,0.559966,0.605607,0.368199,0.326829,0.326829,0.793103,0.428030,0.714564,0.428030,T1000_itrr500_th100_topR5


In [128]:
dfRes.query(f'{s_RF10} and RunType=="Boruta_True" and {s_RF10} and brt_md==7').sort_values('balanced_accuracy',ascending=False)

,MainDataset,Title,site,date,classifier,RunType,cv,FeatureSet,brt_nTrue,brt_nTop,brt_md,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_params
251,dfcmpltPreLC_SITE3,dfcmpltPreLC_SITE3_Boruta_True,SAN,2024-05-01 00:33:31.798267,RandomForestClassifier,Boruta_True,5,NaN,7.0,11.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.984615,0.990000,1.000000,0.989474,0.980000,0.980000,1.000000,1.000000,0.950000,1.000000,T1000_itrr500_th100_topR5
251,dfcmpltPreLC_SITE4,dfcmpltPreLC_SITE4_Boruta_True,RIO,2024-05-01 00:51:18.989204,RandomForestClassifier,Boruta_True,5,NaN,11.0,15.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.987298,0.963333,0.995641,0.992453,1.000000,1.000000,0.926667,0.985185,1.000000,0.985185,T1000_itrr500_th100_topR5
251,dfcmpltPreLC_SITE1,dfcmpltPreLC_SITE1_Boruta_True,LDN,2024-05-01 00:21:09.555851,RandomForestClassifier,Boruta_True,5,NaN,7.0,10.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.948485,0.940714,0.971429,0.931429,0.910000,0.910000,0.971429,0.960000,0.946429,0.960000,T1000_itrr500_th100_topR5
251,dfcmpltPreLC_SITE7,dfcmpltPreLC_SITE7_Boruta_True,NorthAmerica,2024-05-01 01:30:31.348268,RandomForestClassifier,Boruta_True,5,NaN,6.0,10.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.700070,0.655854,0.727854,0.545085,0.492157,0.492157,0.819551,0.611159,0.737375,0.611159,T1000_itrr500_th100_topR5
258,dfcmpltPreLC,dfcmpltPreLC_Boruta_True,Global,2024-05-01 00:09:32.529785,RandomForestClassifier,Boruta_True,5,NaN,7.0,12.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.669375,0.643431,0.704456,0.554342,0.513509,0.513509,0.773354,0.603516,0.704120,0.603516,T1000_itrr500_th100_topR5
251,dfcmpltPreLC_SITE2,dfcmpltPreLC_SITE2_Boruta_True,MTL,2024-05-01 00:24:37.613991,RandomForestClassifier,Boruta_True,5,NaN,2.0,6.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.656897,0.564433,0.598930,0.365249,0.316366,0.316366,0.812500,0.434052,0.722382,0.434052,T1000_itrr500_th100_topR5
251,dfcmpltPreLC_SITE6,dfcmpltPreLC_SITE6_Boruta_True,Canada,2024-05-01 01:15:47.719266,RandomForestClassifier,Boruta_True,5,NaN,2.0,6.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.643750,0.559966,0.605607,0.368199,0.326829,0.326829,0.793103,0.428030,0.714564,0.428030,T1000_itrr500_th100_topR5
251,dfcmpltPreLC_SITE5,dfcmpltPreLC_SITE5_Boruta_True,LUS,2024-05-01 01:02:16.744682,RandomForestClassifier,Boruta_True,5,NaN,1.0,5.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.701105,0.484628,0.448674,0.115693,0.092727,0.092727,0.876529,0.155476,0.769810,0.155476,T1000_itrr500_th100_topR5


In [124]:
s_query = f'{s_CMPLT} and param_clf__n_estimators==1000 and param_clf__max_depth==10 {s_brt}'
dfTmp = deepcopy(dfRes.query(s_query)).sort_values('balanced_accuracy',ascending=False)
print(s_query)
print(f"Max balanced accuracy {dfTmp['balanced_accuracy'].max()}")
# print(f"Max balanced accuracy {dfTmp['balanced_accuracy'].max()}")

dfTmp[lPrettyCols]

MainDataset=="dfcmpltPreLC" and param_clf__n_estimators==1000 and param_clf__max_depth==10 and (RunType=="Boruta_True" or RunType=="Boruta_Top")
Max balanced accuracy 0.7160697645543891


,MainDataset,RunType,classifier,brt_nTrue,brt_nTop,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_md,brt_params
135,dfcmpltPreLC,Boruta_Top,RandomForestClassifier,33.0,37.0,0.757248,0.716070,0.824105,0.625939,0.509219,0.509219,0.922921,0.818090,0.738391,0.818090,3.0,T1000_itrr500_th100_topR5
94,dfcmpltPreLC,Boruta_True,RandomForestClassifier,33.0,37.0,0.751818,0.712712,0.817567,0.623722,0.516037,0.516037,0.909387,0.793453,0.738285,0.793453,3.0,T1000_itrr500_th100_topR5
217,dfcmpltPreLC,Boruta_Top,RandomForestClassifier,16.0,20.0,0.744550,0.711091,0.802736,0.629768,0.543029,0.543029,0.879152,0.750620,0.742452,0.750620,5.0,T1000_itrr500_th100_topR5
299,dfcmpltPreLC,Boruta_Top,RandomForestClassifier,7.0,12.0,0.733677,0.709168,0.786101,0.637572,0.586006,0.586006,0.832331,0.700437,0.750933,0.700437,7.0,T1000_itrr500_th100_topR5
176,dfcmpltPreLC,Boruta_True,RandomForestClassifier,16.0,20.0,0.735483,0.707276,0.789880,0.630308,0.565577,0.565577,0.848975,0.715571,0.746050,0.715571,5.0,T1000_itrr500_th100_topR5
146,dfcmpltPreLC,Boruta_Top,XGBClassifier,33.0,37.0,0.716487,0.699357,0.774260,0.633783,0.613228,0.613228,0.785486,0.656256,0.752713,0.656256,3.0,T1000_itrr500_th100_topR5
156,dfcmpltPreLC,Boruta_Top,XGBClassifier,33.0,37.0,0.705611,0.688002,0.766559,0.620088,0.599642,0.599642,0.776361,0.642255,0.743738,0.642255,3.0,T1000_itrr500_th100_topR5
166,dfcmpltPreLC,Boruta_Top,XGBClassifier,33.0,37.0,0.701983,0.686108,0.747364,0.619812,0.606359,0.606359,0.765858,0.634206,0.744383,0.634206,3.0,T1000_itrr500_th100_topR5
105,dfcmpltPreLC,Boruta_True,XGBClassifier,33.0,37.0,0.701074,0.683469,0.755659,0.614789,0.595123,0.595123,0.771816,0.635852,0.740365,0.635852,3.0,T1000_itrr500_th100_topR5
125,dfcmpltPreLC,Boruta_True,XGBClassifier,33.0,37.0,0.695656,0.678286,0.735493,0.608355,0.590679,0.590679,0.765892,0.628271,0.737077,0.628271,3.0,T1000_itrr500_th100_topR5


In [125]:
# dfTmp.query('classifier=="RandomForestClassifier" and (RunType=="Boruta_True" or RunType=="Boruta_Top")')
dfTmp.query(
    'classifier=="RandomForestClassifier"'
)

,MainDataset,Title,site,date,classifier,RunType,cv,FeatureSet,brt_nTrue,brt_nTop,brt_md,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_params
135,dfcmpltPreLC,dfcmpltPreLC_Boruta_Top,Global,2024-05-01 00:09:32.529785,RandomForestClassifier,Boruta_Top,5,NaN,33.0,37.0,3.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.757248,0.716070,0.824105,0.625939,0.509219,0.509219,0.922921,0.818090,0.738391,0.818090,T1000_itrr500_th100_topR5
94,dfcmpltPreLC,dfcmpltPreLC_Boruta_True,Global,2024-05-01 00:09:32.529785,RandomForestClassifier,Boruta_True,5,NaN,33.0,37.0,3.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.751818,0.712712,0.817567,0.623722,0.516037,0.516037,0.909387,0.793453,0.738285,0.793453,T1000_itrr500_th100_topR5
217,dfcmpltPreLC,dfcmpltPreLC_Boruta_Top,Global,2024-05-01 00:09:32.529785,RandomForestClassifier,Boruta_Top,5,NaN,16.0,20.0,5.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.744550,0.711091,0.802736,0.629768,0.543029,0.543029,0.879152,0.750620,0.742452,0.750620,T1000_itrr500_th100_topR5
299,dfcmpltPreLC,dfcmpltPreLC_Boruta_Top,Global,2024-05-01 00:09:32.529785,RandomForestClassifier,Boruta_Top,5,NaN,7.0,12.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.733677,0.709168,0.786101,0.637572,0.586006,0.586006,0.832331,0.700437,0.750933,0.700437,T1000_itrr500_th100_topR5
176,dfcmpltPreLC,dfcmpltPreLC_Boruta_True,Global,2024-05-01 00:09:32.529785,RandomForestClassifier,Boruta_True,5,NaN,16.0,20.0,5.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.735483,0.707276,0.789880,0.630308,0.565577,0.565577,0.848975,0.715571,0.746050,0.715571,T1000_itrr500_th100_topR5
258,dfcmpltPreLC,dfcmpltPreLC_Boruta_True,Global,2024-05-01 00:09:32.529785,RandomForestClassifier,Boruta_True,5,NaN,7.0,12.0,7.0,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.669375,0.643431,0.704456,0.554342,0.513509,0.513509,0.773354,0.603516,0.704120,0.603516,T1000_itrr500_th100_topR5


In [126]:
dfTmp.query('classifier=="XGBClassifier"')

,MainDataset,Title,site,date,classifier,RunType,cv,FeatureSet,brt_nTrue,brt_nTop,brt_md,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_params
146,dfcmpltPreLC,dfcmpltPreLC_Boruta_Top,Global,2024-05-01 00:09:32.529785,XGBClassifier,Boruta_Top,5,NaN,33.0,37.0,3.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.716487,0.699357,0.774260,0.633783,0.613228,0.613228,0.785486,0.656256,0.752713,0.656256,T1000_itrr500_th100_topR5
156,dfcmpltPreLC,dfcmpltPreLC_Boruta_Top,Global,2024-05-01 00:09:32.529785,XGBClassifier,Boruta_Top,5,NaN,33.0,37.0,3.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.2,0.705611,0.688002,0.766559,0.620088,0.599642,0.599642,0.776361,0.642255,0.743738,0.642255,T1000_itrr500_th100_topR5
166,dfcmpltPreLC,dfcmpltPreLC_Boruta_Top,Global,2024-05-01 00:09:32.529785,XGBClassifier,Boruta_Top,5,NaN,33.0,37.0,3.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,1.0,0.701983,0.686108,0.747364,0.619812,0.606359,0.606359,0.765858,0.634206,0.744383,0.634206,T1000_itrr500_th100_topR5
105,dfcmpltPreLC,dfcmpltPreLC_Boruta_True,Global,2024-05-01 00:09:32.529785,XGBClassifier,Boruta_True,5,NaN,33.0,37.0,3.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.701074,0.683469,0.755659,0.614789,0.595123,0.595123,0.771816,0.635852,0.740365,0.635852,T1000_itrr500_th100_topR5
125,dfcmpltPreLC,dfcmpltPreLC_Boruta_True,Global,2024-05-01 00:09:32.529785,XGBClassifier,Boruta_True,5,NaN,33.0,37.0,3.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,1.0,0.695656,0.678286,0.735493,0.608355,0.590679,0.590679,0.765892,0.628271,0.737077,0.628271,T1000_itrr500_th100_topR5
115,dfcmpltPreLC,dfcmpltPreLC_Boruta_True,Global,2024-05-01 00:09:32.529785,XGBClassifier,Boruta_True,5,NaN,33.0,37.0,3.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.2,0.693830,0.677101,0.747146,0.607984,0.592901,0.592901,0.761301,0.624301,0.736837,0.624301,T1000_itrr500_th100_topR5
320,dfcmpltPreLC,dfcmpltPreLC_Boruta_Top,Global,2024-05-01 00:09:32.529785,XGBClassifier,Boruta_Top,5,NaN,7.0,12.0,7.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.2,0.687507,0.670340,0.724706,0.598936,0.583810,0.583810,0.756869,0.616816,0.731815,0.616816,T1000_itrr500_th100_topR5
310,dfcmpltPreLC,dfcmpltPreLC_Boruta_Top,Global,2024-05-01 00:09:32.529785,XGBClassifier,Boruta_Top,5,NaN,7.0,12.0,7.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.686598,0.669564,0.732088,0.598146,0.583784,0.583784,0.755343,0.614909,0.731462,0.614909,T1000_itrr500_th100_topR5
228,dfcmpltPreLC,dfcmpltPreLC_Boruta_Top,Global,2024-05-01 00:09:32.529785,XGBClassifier,Boruta_Top,5,NaN,16.0,20.0,5.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,0.1,0.686561,0.668369,0.729360,0.594992,0.576941,0.576941,0.759797,0.615006,0.729583,0.615006,T1000_itrr500_th100_topR5
330,dfcmpltPreLC,dfcmpltPreLC_Boruta_Top,Global,2024-05-01 00:09:32.529785,XGBClassifier,Boruta_Top,5,NaN,7.0,12.0,7.0,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,1.0,0.676631,0.662022,0.712460,0.592483,0.588330,0.588330,0.735714,0.598543,0.728434,0.598543,T1000_itrr500_th100_topR5


# Difference between Boruta Selected Datasets

In [21]:
lbrt_7 = MPutils.getTrueFeatList(
    "OUTPUT/MP/05-classifiers/DataV3/dfcmplt/FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD7.csv"
)

lbrt_5 = MPutils.getTrueFeatList(
    "OUTPUT/MP/05-classifiers/DataV3/dfcmplt/FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD5.csv"
)

lbrt_3 = MPutils.getTrueFeatList(
    "OUTPUT/MP/05-classifiers/DataV3/dfcmplt/FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD3.csv"
)

len(lbrt_7), len(lbrt_5), len(lbrt_3)

(68, 81, 113)

In [24]:
set(lbrt_7).difference(set(lbrt_5))

set()

In [27]:
set(lbrt_5).difference(set(lbrt_7))

{'CD-covid_test_result_2',
 'CD-gcs',
 'CD-neurologic-other-1',
 'CD-neurologic-other-4',
 'CD-required_oxygen',
 'CD-respiratory_rate',
 'CD-severity_level___2',
 'LABS-IND-basophil_count',
 'LABS-IND-eosinophil',
 'LABS-IND-pct',
 'SYMPT-fever_38_0oc___2',
 'SYMPT-night_sweats___2',
 'SYMPT-runny_nose_rhinorrhea___2'}

In [32]:
set(lbrt_7).difference(set(lbrt_3))

set()

In [33]:
set(lbrt_3).difference(set(lbrt_5))

{'CD-covid_test_result_1',
 'CD-covid_test_result_3',
 'CD-hospitalized',
 'CD-insomnia___1',
 'CD-medication_received___4',
 'CD-medication_received___5',
 'CD-memory_problems___1',
 'CD-respiratory-other-1',
 'CD-severity_level___3',
 'CD-type_of_care___2',
 'HHX-Arterial Hypertension',
 'HHX-Diabetes',
 'HHX-OTHER-Sleep-disorder',
 'HHX-Obesity',
 'IMG-ctt',
 'LAB-test-lung_function',
 'LABS-IND-cpk',
 'LABS-IND-inr',
 'MED-Cardiovascular-agents',
 'MED-Respiratory-tract-agents',
 'SYMPT-cough___1',
 'SYMPT-diarrhea___1',
 'SYMPT-extremity_weakness_or_numb___1',
 'SYMPT-fatigue___1',
 'SYMPT-fever_38_0oc___1',
 'SYMPT-joint_pain_arthralgia___1',
 'SYMPT-muscle_aches_myalgia___1',
 'SYMPT-nausea_vomiting___1',
 'SYMPT-runny_nose_rhinorrhea___1',
 'SYMPT-sore_throat___1',
 'SYMPT-trouble_speaking_aphasia_d___2',
 'SYMPT-weight_loss___1'}

In [34]:
set(lbrt_3).difference(set(lbrt_5))

{'CD-covid_test_result_1',
 'CD-covid_test_result_3',
 'CD-hospitalized',
 'CD-insomnia___1',
 'CD-medication_received___4',
 'CD-medication_received___5',
 'CD-memory_problems___1',
 'CD-respiratory-other-1',
 'CD-severity_level___3',
 'CD-type_of_care___2',
 'HHX-Arterial Hypertension',
 'HHX-Diabetes',
 'HHX-OTHER-Sleep-disorder',
 'HHX-Obesity',
 'IMG-ctt',
 'LAB-test-lung_function',
 'LABS-IND-cpk',
 'LABS-IND-inr',
 'MED-Cardiovascular-agents',
 'MED-Respiratory-tract-agents',
 'SYMPT-cough___1',
 'SYMPT-diarrhea___1',
 'SYMPT-extremity_weakness_or_numb___1',
 'SYMPT-fatigue___1',
 'SYMPT-fever_38_0oc___1',
 'SYMPT-joint_pain_arthralgia___1',
 'SYMPT-muscle_aches_myalgia___1',
 'SYMPT-nausea_vomiting___1',
 'SYMPT-runny_nose_rhinorrhea___1',
 'SYMPT-sore_throat___1',
 'SYMPT-trouble_speaking_aphasia_d___2',
 'SYMPT-weight_loss___1'}

In [35]:
set(lbrt_5).difference(set(lbrt_3))

set()

In [30]:
set(lbrt_5).difference(set(lbrt_7))

{'CD-covid_test_result_2',
 'CD-gcs',
 'CD-neurologic-other-1',
 'CD-neurologic-other-4',
 'CD-required_oxygen',
 'CD-respiratory_rate',
 'CD-severity_level___2',
 'LABS-IND-basophil_count',
 'LABS-IND-eosinophil',
 'LABS-IND-pct',
 'SYMPT-fever_38_0oc___2',
 'SYMPT-night_sweats___2',
 'SYMPT-runny_nose_rhinorrhea___2'}

# Validation Results

In [39]:
dfVal = dfRes.query(f'{s_RF10} and {s_val}').sort_values('balanced_accuracy',ascending=False)
dfVal

,MainDataset,Title,site,date,classifier,RunType,cv,FeatureSet,brt_nTrue,brt_nTop,brt_md,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_params
5,dfcmpltPreLC3_SITE4,dfcmpltPreLC3_SITE4_HoldoutVal,RIO,2024-05-03 16:51:36.678966,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_SITE4_Boruta_T1000_itrr500_th...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,NaN
5,dfcmpltPreLC3_SITE4,dfcmpltPreLC3_SITE4_HoldoutVal,RIO,2024-05-03 16:51:34.526319,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.987097,0.960000,0.976154,0.992453,1.000000,1.000000,0.920000,0.985185,1.000000,0.985185,NaN
5,dfcmpltPreLC3_SITE1,dfcmpltPreLC3_SITE1_HoldoutVal,LDN,2024-05-03 16:51:18.522883,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.877273,0.878571,0.937143,0.858608,0.900000,0.900000,0.857143,0.845000,0.941667,0.845000,NaN
5,dfcmpltPreLC3_SITE1,dfcmpltPreLC3_SITE1_HoldoutVal,LDN,2024-05-03 16:51:21.130977,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_SITE1_Boruta_T1000_itrr500_th...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.842424,0.849286,0.921429,0.808485,0.870000,0.870000,0.828571,0.770000,0.916667,0.770000,NaN
5,dfcmpltPreLC3_SITE3,dfcmpltPreLC3_SITE3_HoldoutVal,SAN,2024-05-03 16:51:30.094372,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.819780,0.707576,0.895152,0.879135,0.881818,0.881818,0.533333,0.891708,0.550000,0.891708,NaN
5,dfcmpltPreLC3,dfcmpltPreLC3_HoldoutVal,Global,2024-05-03 16:51:15.446652,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.724932,0.701011,0.782271,0.628644,0.583464,0.583464,0.818558,0.686126,0.748051,0.686126,NaN
5,dfcmpltPreLC3_SITE7,dfcmpltPreLC3_SITE7_HoldoutVal,NorthAmerica,2024-05-03 16:51:54.408983,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_SITE7_Boruta_T1000_itrr500_th...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.757477,0.696049,0.786300,0.583824,0.474745,0.474745,0.917353,0.761414,0.756088,0.761414,NaN
5,dfcmpltPreLC3_SITE3,dfcmpltPreLC3_SITE3_HoldoutVal,SAN,2024-05-03 16:51:32.198611,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_SITE3_Boruta_T1000_itrr500_th...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.808791,0.692727,0.898788,0.877295,0.885455,0.885455,0.500000,0.875000,0.580000,0.875000,NaN
5,dfcmpltPreLC3_SITE6,dfcmpltPreLC3_SITE6_HoldoutVal,Canada,2024-05-03 16:51:48.127120,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_SITE6_Boruta_T1000_itrr500_th...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.725559,0.633772,0.730491,0.470021,0.385000,0.385000,0.882545,0.607125,0.756793,0.607125,NaN
5,dfcmpltPreLC3_SITE7,dfcmpltPreLC3_SITE7_HoldoutVal,NorthAmerica,2024-05-03 16:51:51.246566,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.690426,0.628921,0.733176,0.486470,0.407529,0.407529,0.850312,0.616585,0.717630,0.616585,NaN


In [40]:
dfVal = dfVal.sort_values(['FeatureSet','MainDataset'])
dfVal

,MainDataset,Title,site,date,classifier,RunType,cv,FeatureSet,brt_nTrue,brt_nTop,brt_md,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV,brt_params
5,dfcmpltPreLC3,dfcmpltPreLC3_HoldoutVal,Global,2024-05-03 16:51:15.446652,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.724932,0.701011,0.782271,0.628644,0.583464,0.583464,0.818558,0.686126,0.748051,0.686126,NaN
5,dfcmpltPreLC3_SITE1,dfcmpltPreLC3_SITE1_HoldoutVal,LDN,2024-05-03 16:51:18.522883,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.877273,0.878571,0.937143,0.858608,0.900000,0.900000,0.857143,0.845000,0.941667,0.845000,NaN
5,dfcmpltPreLC3_SITE2,dfcmpltPreLC3_SITE2_HoldoutVal,MTL,2024-05-03 16:51:24.594688,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.688021,0.572072,0.678886,0.347263,0.269524,0.269524,0.874620,0.491008,0.728607,0.491008,NaN
5,dfcmpltPreLC3_SITE3,dfcmpltPreLC3_SITE3_HoldoutVal,SAN,2024-05-03 16:51:30.094372,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.819780,0.707576,0.895152,0.879135,0.881818,0.881818,0.533333,0.891708,0.550000,0.891708,NaN
5,dfcmpltPreLC3_SITE4,dfcmpltPreLC3_SITE4_HoldoutVal,RIO,2024-05-03 16:51:34.526319,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.987097,0.960000,0.976154,0.992453,1.000000,1.000000,0.920000,0.985185,1.000000,0.985185,NaN
5,dfcmpltPreLC3_SITE5,dfcmpltPreLC3_SITE5_HoldoutVal,LUS,2024-05-03 16:51:39.189665,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.751190,0.509621,0.581856,0.101667,0.072727,0.072727,0.946515,0.170000,0.779730,0.170000,NaN
5,dfcmpltPreLC3_SITE6,dfcmpltPreLC3_SITE6_HoldoutVal,Canada,2024-05-03 16:51:45.149447,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.700300,0.599135,0.719008,0.407044,0.325000,0.325000,0.873269,0.546249,0.737166,0.546249,NaN
5,dfcmpltPreLC3_SITE7,dfcmpltPreLC3_SITE7_HoldoutVal,NorthAmerica,2024-05-03 16:51:51.246566,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_Boruta_T1000_itrr500_th100_to...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.690426,0.628921,0.733176,0.486470,0.407529,0.407529,0.850312,0.616585,0.717630,0.616585,NaN
5,dfcmpltPreLC3_SITE1,dfcmpltPreLC3_SITE1_HoldoutVal,LDN,2024-05-03 16:51:21.130977,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_SITE1_Boruta_T1000_itrr500_th...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.842424,0.849286,0.921429,0.808485,0.870000,0.870000,0.828571,0.770000,0.916667,0.770000,NaN
5,dfcmpltPreLC3_SITE2,dfcmpltPreLC3_SITE2_HoldoutVal,MTL,2024-05-03 16:51:27.597787,RandomForestClassifier,HoldoutVal,5,FS_dfcmpltPreLC3_SITE2_Boruta_T1000_itrr500_th...,NaN,NaN,NaN,RandomForestClassifier(),10.0,1000.0,42.0,NaN,NaN,0.705322,0.595425,0.670377,0.391204,0.308730,0.308730,0.882120,0.537945,0.741185,0.537945,NaN


In [41]:
# # Create the ExcelWriter object

sGlobal = 'FS_dfcmpltPreLC3_Boruta_T1000_itrr500_th100_topR5_MD3'

writer = pd.ExcelWriter('../OUTPUTS/prelim_preLongCOVID3_Val.xlsx', engine='xlsxwriter')
dfVal[lPrettyColsSite+['brt_md','brt_nTrue','Title','FeatureSet']].sort_values('balanced_accuracy',ascending=False).to_excel(writer,sheet_name='Results')


for i,d in pd.DataFrame(dfVal.sort_values([])).iterrows():
    tmpFS = d['FeatureSet']
    if tmpFS == sGlobal:
        sheetName = f'Global Feat - {d["site"]}'
    else:
        sheetName = f'Site Feat - {d["site"]}'
        
    tmpMainData = d['MainDataset']
    
    dFeat = pd.read_csv(f'OUTPUT/MP/05-classifiers/DataV3/{tmpMainData}/FR_{tmpMainData}_HoldoutVal_{tmpFS}.csv')
    # f = f'OUTPUT/MP/05-classifiers/DataV3/{tmpdataset}/FS_{tmpdataset}_Boruta_T1000_itrr500_th100_topR5_MD{md}.csv'
    # dfeat = pd.read_csv(f)
    
    # MPutils.getTrueFeatList(f,asDF=True).to_excel(writer, sheet_name=d['site'])
    dFeat.to_excel(writer,sheet_name=sheetName)


    # Write each dataframe to a different sheet
    # df1.to_excel(writer, sheet_name=d['site'])

#     # Save the Excel file
writer.close()


# DEPRECIATED - May 2

# Validation Result

- lCmplt = MPutils.getTrueFeatList("OUTPUT/MP/05-classifiers/DataV3/dfcmplt/FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD7.csv")


- lColFS = lCmplt
- selFeatName = "FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD7"

In [36]:
dfValRes = pd.read_csv(
    "OUTPUT/MP/05-classifiers/DataV3/CLFHoldoutValRunCombined_FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD7_selCol.csv"
)
dfValRes.columns = [x.replace("mean_test_", "") for x in dfValRes.columns]

In [37]:
dfValRes

,MainDataset,Title,date,classifier,RunType,cv,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,param_clf__learning_rate,accuracy,balanced_accuracy,roc_auc,f1,recall,sensitivity,specificity,precision,NPV,PPV
0,dfcmplt,dfcmplt_HoldoutVal,2024-04-30 16:56:56.476517,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),3.0,100.0,42.0,NaN,NaN,0.884854,0.858958,0.963452,0.835139,0.728498,0.728498,0.989417,0.979101,0.845081,0.979101
1,dfcmplt,dfcmplt_HoldoutVal,2024-04-30 16:56:56.476517,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),3.0,1000.0,42.0,NaN,NaN,0.883945,0.857808,0.964534,0.833637,0.726200,0.726200,0.989417,0.979183,0.843980,0.979183
2,dfcmplt,dfcmplt_HoldoutVal,2024-04-30 16:56:56.476517,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),6.0,100.0,42.0,NaN,NaN,0.921107,0.906113,0.978601,0.893591,0.830363,0.830363,0.981864,0.969058,0.896988,0.969058
3,dfcmplt,dfcmplt_HoldoutVal,2024-04-30 16:56:56.476517,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),6.0,1000.0,42.0,NaN,NaN,0.913875,0.897112,0.979010,0.882865,0.812360,0.812360,0.981864,0.968757,0.887128,0.968757
4,dfcmplt,dfcmplt_HoldoutVal,2024-04-30 16:56:56.476517,RandomForestClassifier,HoldoutVal,5,RandomForestClassifier(),10.0,100.0,42.0,NaN,NaN,0.940156,0.931743,0.982863,0.922105,0.889198,0.889198,0.974288,0.958830,0.929950,0.958830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,dfcmplt_SITE7,dfcmplt_SITE7_HoldoutVal,2024-04-30 16:57:17.132314,XGBClassifier,HoldoutVal,5,"XGBClassifier(base_score=None, booster=None, c...",10.0,1000.0,42.0,NaN,1.0,0.948996,0.938437,0.973932,0.927617,0.899246,0.899246,0.977628,0.959180,0.944480,0.959180
324,dfcmplt_SITE7,dfcmplt_SITE7_HoldoutVal,2024-04-30 16:57:17.132314,XGBClassifier,HoldoutVal,5,"XGBClassifier(base_score=None, booster=None, c...",15.0,100.0,42.0,NaN,1.0,0.954670,0.943794,0.978004,0.935657,0.903243,0.903243,0.984345,0.971324,0.946611,0.971324
325,dfcmplt_SITE7,dfcmplt_SITE7_HoldoutVal,2024-04-30 16:57:17.132314,XGBClassifier,HoldoutVal,5,"XGBClassifier(base_score=None, booster=None, c...",15.0,1000.0,42.0,NaN,1.0,0.956078,0.946516,0.971732,0.937991,0.910935,0.910935,0.982097,0.967575,0.950608,0.967575
326,dfcmplt_SITE7,dfcmplt_SITE7_HoldoutVal,2024-04-30 16:57:17.132314,XGBClassifier,HoldoutVal,5,"XGBClassifier(base_score=None, booster=None, c...",NaN,100.0,42.0,NaN,1.0,0.948996,0.937675,0.977302,0.927666,0.895475,0.895475,0.979875,0.962652,0.942180,0.962652


In [41]:
site_name_dict

{1: 'LDN', 2: 'MTL', 3: 'SAN', 4: 'RIO', 5: 'LUS', 6: 'CA', 7: 'NA'}

In [45]:
siteMap = {
    "dfcmplt": "Global",
    "dfcmplt_SITE1": "LDN",
    "dfcmplt_SITE2": "MTL",
    "dfcmplt_SITE3": "SAN",
    "dfcmplt_SITE4": "RIO",
    "dfcmplt_SITE5": "LUS",
    "dfcmplt_SITE6": "Canada",
    "dfcmplt_SITE7": "NorthAmerica",
}

In [47]:
dfValRes['MainDataset'] = dfValRes['MainDataset'].map(siteMap)

# Validation Results -- Final?

In [49]:
dfValRes.query(f'classifier=="RandomForestClassifier" and param_clf__max_depth==10 and param_clf__n_estimators==1000')[[x for x in lPrettyCols if 'brt_' not in x]].T

,5,46,87,128,169,210,251,292
MainDataset,Global,LDN,MTL,SAN,RIO,LUS,Canada,NorthAmerica
RunType,HoldoutVal,HoldoutVal,HoldoutVal,HoldoutVal,HoldoutVal,HoldoutVal,HoldoutVal,HoldoutVal
classifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier,RandomForestClassifier
accuracy,0.94197,0.912121,0.979369,0.969231,0.993548,0.829932,0.967151,0.964589
balanced_accuracy,0.933624,0.912143,0.967117,0.980909,0.98,0.66486,0.956518,0.958143
roc_auc,0.983206,0.951429,0.994132,1.0,1.0,0.861153,0.98919,0.990181
f1,0.924669,0.892929,0.965854,0.97995,0.996226,0.480784,0.947304,0.95083
recall,0.891445,0.91,0.934234,0.961818,1.0,0.367273,0.926829,0.934163
sensitivity,0.891445,0.91,0.934234,0.961818,1.0,0.367273,0.926829,0.934163
specificity,0.975803,0.914286,1.0,1.0,0.96,0.962447,0.986207,0.982122


# Feature Ranking Results

In [51]:
dfFR = pd.read_csv('OUTPUT/MP/05-classifiers/DataV3/dfcmplt/FR_dfcmplt_HoldoutValFS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD7.csv')

In [52]:
dfFR

,Rank,Feature,Importance
0,1,SYMPT-fatigue___2,0.1659
1,2,SYMPT-shortness_of_breath_dyspne___2,0.1153
2,3,SYMPT-loss_of_taste_lost_of_smel___2,0.0566
3,4,SYMPT-cough___2,0.0501
4,5,LABS-IND-triglycerides,0.0466
...,...,...,...
63,64,LABS-IND-basophil,0.0011
64,65,SYMPT-wheezing_or_stridor___2,0.0010
65,66,SYMPT-ear_pain___2,0.0009
66,66,CD-adjunctive___1,0.0009


In [57]:
dfFR[dfFR['Importance']>0.01]

,Rank,Feature,Importance
0,1,SYMPT-fatigue___2,0.1659
1,2,SYMPT-shortness_of_breath_dyspne___2,0.1153
2,3,SYMPT-loss_of_taste_lost_of_smel___2,0.0566
3,4,SYMPT-cough___2,0.0501
4,5,LABS-IND-triglycerides,0.0466
5,6,SYMPT-joint_pain_arthralgia___2,0.0444
6,7,SYMPT-muscle_aches_myalgia___2,0.0432
7,8,LABS-IND-monocyte_count,0.0410
8,9,SYMPT-headache___2,0.0384
9,10,LABS-IND-bilirubin,0.0356
